Этот код используется для составления SQL-запроса, который позволяет оценить процент сайтов, попадающих в заранее определенные категории, используя данные, полученные с помощью указанных сервисов. Запрос формирует условия для классификации доменов по категориям, сравнивая значения в столбце req_host с регулярными выражениями, связанными с каждой категорией. Статистика по категориям помогает оценить, насколько эффективно проделанная работа по категоризации позволяет в будущем классифицировать сайты с использованием этого запроса. Код включает в себя агрегацию данных по категориям, генерацию SQL-запроса и вывод результата для анализа.

1. Загрузка библиотек: Импортируются необходимые библиотеки для работы с данными (pandas), обработки паролей (getpass), работы с операционной системой (os), а также для работы с JSON и строками (json, ast).

2. Выгрузка данных: Загружается CSV файл combined_hosts_with_fixed_regex_v4.csv в DataFrame df_reg. Далее с помощью метода value_counts() подсчитываются уникальные значения в столбце 'cat_popular'.

3. Агрегация данных: Данные группируются по столбцу 'cat_popular', и для каждой категории собираются соответствующие значения из столбца 'url_regex_mod' в виде списка. Результат сохраняется в новый DataFrame df_reg_unite.

4. Формирование SQL запроса: Создается строка SQL-запроса, начиная с подзапроса, в котором создаются условия для классификации req_host по категориям. Для каждой категории из DataFrame df_reg_unite формируются условия WHEN для функции match(), которая проверяет соответствие доменов в req_host. Если совпадение найдено, то присваивается соответствующая категория.

5. Заполнение и завершение строки запроса: Для каждой категории добавляются строки с условиями WHEN, где домены (из списка в df_reg_unite) добавляются с помощью метода join(). Результирующая строка расширяется с каждым условием и завершается дополнительными операциями, такими как фильтрация по дате и времени, а также групировка и сортировка по категориям. Запрос выводится в консоль с помощью print().

1. Загрузка библиотек

In [1]:
import pandas as pd
import getpass
import os
import json
import ast

2. Выгрузка данных

In [2]:
df_reg = pd.read_csv('combined_hosts_with_fixed_regex_v40.csv')


3. Агрегация данных

In [3]:
df_reg_unite = df_reg.groupby('cat_popular').agg(
    {'url_regex_mod': list}
).reset_index()
df_reg_unite

,cat_popular,url_regex_mod
0,Advertisements.,"[^([\w-]+\.)*(www\.)?mxptint\.(net)$, ^([\w-]+..."
1,Business/Economy (alphaMountain.ai),[^([\w-]+\.)*(www\.)?xn--80aebkobnwfcnsfk1e0h\...
2,Malicious (alphaMountain.ai),[34.160.111.145]
3,Phishing and Other Frauds,[^([\w-]+\.)*(www\.)?dolphin-anty-mirror\.(org)$]
4,Suspicious (alphaMountain.ai),[146.190.27.215]
...,...,...
174,web analytics,"[^([\w-]+\.)*(www\.)?digitaltarget\.(ru)$, 87...."
175,web hosting,"[^([\w-]+\.)*(www\.)?com\.(ua)$, ^([\w-]+\.)*(..."
176,web infrastructure,"[^([\w-]+\.)*(www\.)?kimberlite\.(io)$, ^([\w-..."
177,webmail,"[^([\w-]+\.)*(www\.)?mail\.(ru)$, ^([\w-]+\.)*..."


4. Формирование SQL запроса:

In [4]:
string = '''with a as (
	select
		cn, 
		proxy_ip,
		proxy_port,
		ts_millis,
		proxy_service, --group
		error, --count
		tx_bytes, --sum
		rx_bytes, --sum
		req_host, --count(distinct)
        CASE \n'''


for cat, regs in df_reg_unite.values:
    string += "\t\t\tWHEN match(req_host, '"
    string += "') +\n\t\t\t\t match(req_host, '".join(regs)
    string += f"') > 0 THEN '{cat}'\n"
    
string += """		ELSE 'other' END req_category2
        from -- ваша база данных 
		where 1=1
			and toYYYYMM(ts_millis) = toYYYYMM(toDate('2021-09-18'))
			and ts_millis > toDateTime('2021-09-18') 
			and ts_millis <= addHours(toDateTime('2021-09-18'), 24)
)
select 
	req_category2,
	count(*) as cnt
from a
group by 
	req_category2
order by 
	cnt desc
"""

5. Заполнение строки запроса и завершение запроса 

Этот код я использую для запроса в clickhouse. На основе полученного результата запроса, смотрю, сколько процентов занимает категория others

In [5]:
string = '''with a as (
    select
        cn, 
        proxy_ip,
        proxy_port,
        ts_millis,
        proxy_service, --group
        error, --count
        tx_bytes, --sum
        rx_bytes, --sum
        req_host, --count(distinct)
        CASE \n'''


for cat, regs in df_reg_unite.values:
    string += f"{' '*12}WHEN multiMatchAny(req_host, [\n{' '*16}'"
    string += f"',\n{' '*16}'".join(regs)
    string += f"'\n{' '*12}]) > 0 THEN '{cat}'\n"
    
string += """        ELSE 'other' END req_category2
        from ваша база данных 
        where 1=1
            and toYYYYMM(ts_millis) = toYYYYMM(toDate('2021-09-18'))
            and ts_millis > toDateTime('2021-09-18') 
            and ts_millis <= addHours(toDateTime('2021-09-18'), 24)
)
select 
    req_category2,
    count(*) as cnt
from a
group by 
    req_category2
order by 
    cnt desc
"""
#print(string)

1. Загрузка библиотек: Библиотеки, необходимые для работы с данными и операциями с паролями, операционной системой, JSON и строками, были успешно импортированы. Это обеспечило необходимую функциональность для дальнейшей обработки данных.

2. Выгрузка данных: Данные из CSV файла combined_hosts_with_fixed_regex_v4.csv были загружены в DataFrame. Использование метода value_counts() позволило подсчитать уникальные значения в столбце 'cat_popular', что помогло выявить распределение категорий.

3. Агрегация данных: Данные были сгруппированы по столбцу 'cat_popular', и для каждой категории был собран список значений из столбца 'url_regex_mod'. Это позволило создать новый DataFrame df_reg_unite, в котором каждая категория была представлена списком связанных значений.

4. Формирование SQL запроса: Строка SQL-запроса была сформирована, начиная с подзапроса, в котором создавались условия для классификации req_host по категориям. Для каждой категории из df_reg_unite были добавлены соответствующие условия WHEN, проверяющие соответствие доменов в req_host.

5. Заполнение и завершение строки запроса: Были добавлены строки с условиями WHEN, в которых домены, полученные из списка в df_reg_unite, добавлялись с использованием метода join(). В результате строка запроса была дополнена необходимыми условиями и